# Market Data EDA

As part of this notebook we are going to take a look at the following Kaggle dataset:

https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs

The plan is to analyze this data and perform some quality checks.

In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import dask.dataframe as dd
import pandas_datareader.data as web

pd.set_option('display.min_rows', 50)

/home/test/.conda/envs/trading-strategies/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# Path to data location
DATA_DIR = '../data/price-volume-data-for-all-us-stocks-etfs/Data'
STOCK_DIR = 'Stocks'
ETF_DIR = 'ETFs'

Some basic functions to get stock & etf tickers and extract data from csv files.

In [ ]:
class EquityData:
    """
    Provides an access layer to market data files made available
    as part of the following Kaggle dataset:

        https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs

    Each file contains market data for one ticker (either stock or ETF). Also, each filename
    has a specific naming convention: it consists of a ticker and a suffix (common to all files).

    Attributes:
        stock_dir (str): Directory location for stock files.
        etf_dir (str): Directory location for ETF files.
        file_suffix (str): A file suffix used by stock & ETF files.
    """

    FILE_SUFFIX = '.us.txt'

    def __init__(self, data_dir, stock_dir, etf_dir, file_suffix=FILE_SUFFIX):

        self.stock_dir = os.path.join(data_dir, stock_dir)
        self.etf_dir = os.path.join(data_dir, etf_dir)
        self.file_suffix = file_suffix

    @staticmethod
    def _get_tickers(dirname, file_suffix):
        """
        Creates a list of available tickers in a given directory.

        Only non-empty files are considered.

        Args:
            file_suffix (str): A file suffix used by each data file.

        Returns:
            (List[str]): A list of available tickers.
        """

        tickers = []
        for file in os.listdir(dirname):
            # This dataset includes empty files which we exclude here.
            if os.stat(os.path.join(dirname, file)).st_size > 0:
                tickers.append(file.replace(file_suffix, ''))

        return tickers

    def get_etf_tickers(self):
        """
        Returns a list of available ETF tickers.

        Returns:
            (List[str]): A sorted list of available ETF tickers.
        """

        return sorted(self._get_tickers(self.etf_dir, self.file_suffix))

    def get_stock_tickers(self):
        """
        Returns a list of available stock tickers.

        Returns:
            (List[str]): A sorted list of available stock tickers.
        """
        return sorted(self._get_tickers(self.stock_dir, self.file_suffix))

    def get_all_tickers(self):
        """
        Returns a list of available stock and ETF tickers.

        Returns:
            (List[str]): A sorted list of available stock and ETF tickers.
        """
        return sorted(self.get_etf_tickers() + self.get_stock_tickers())

    def get_csv_data(self, ticker, start_date=None, end_date=None):
        """
        Returns a list of available stock and ETF tickers.

        Args:
            ticker (str): A ticker name.
            start_date (str): A start date
            end_date (str): An end date

        Returns:
            (pd.DataFrame): Parsed market data for a given ticker.
        """
        ticker = ticker.lower()

        # Check ETFs first since the number of potential hits is much smaller.
        if ticker in self.get_etf_tickers():
            dirname = self.etf_dir
        else:
            dirname = self.stock_dir

        result = pd.DataFrame()
        csv = pd.read_csv(os.path.join(dirname, ticker + self.file_suffix))
        result = pd.DataFrame({
            'date': csv['Date'],
            'ticker': ticker,
            'price': csv['Close'],
            'volume': csv['Volume']
        })
        result['date'] = result['date'].map(
            lambda t: datetime.strptime(t, '%Y-%m-%d')).to_numpy()

        if start_date is not None:
            result = result[result['date'] >= start_date]

        if end_date is not None:
            result = result[result['date'] <= end_date]

        result.index = result.date

        # Only including weekdays
        result['dayofweek'] = pd.DatetimeIndex(result.index).dayofweek
        result = result[(result.dayofweek != 5) & (result.dayofweek != 6)]

        return result

In [ ]:
equity_data = EquityData(DATA_DIR, STOCK_DIR, ETF_DIR)

Let's try out this API and load market data for 'SPY' (ETF tracking S&P 500 index)

In [ ]:
example_ticker = 'SPY'

data = equity_data.get_csv_data(example_ticker)
data.head()

Let's make sure there is no overlap between between stock and ETF tickers.

In [ ]:
etf_tickers = equity_data.get_etf_tickers()
stock_tickers = equity_data.get_stock_tickers()

if len(np.intersect1d(etf_tickers, stock_tickers)) == 0:
    print('No overlap - good')
else:
    print('There is some overlap')

Let's calculate some basic stats for each ticker. Given limited resources of my laptop I am going to analyze each file separately and aggregate results. Another approach would be to use Spark or Dask (something that we will try to do in the following section).

In [ ]:
stats = []

for ticker in tqdm(equity_data.get_all_tickers()):
    data = equity_data.get_csv_data(ticker)
    stats.append({
        'ticker':
        ticker,
        'min_date':
        data.index.min(),
        'max_date':
        data.index.max(),
        'dollar_volume':
        np.median(data['price'][-10:] * data['volume'][-10:])
    })

stats = pd.DataFrame(stats)

In [ ]:
cnt_min_date = stats.groupby(
    ['min_date'])['ticker'].count().reset_index().rename({'ticker': 'count'})

Let's calculate the number of data points for each date.

In [ ]:
def count_data_points(ticker_list):
    cnt = None
    for ticker in tqdm(ticker_list):
        data = equity_data.get_csv_data(ticker)
        data['count'] = 1

        if cnt is None:
            cnt = data['count']
        else:
            cnt = cnt.add(data['count'], fill_value=0)

    return cnt

In [ ]:
cnt_etf = count_data_points(equity_data.get_etf_tickers())
cnt_stock = count_data_points(equity_data.get_stock_tickers())

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(cnt_etf.index, cnt_etf.values)
plt.plot(cnt_stock.index, cnt_stock.values)
plt.plot(cnt_min_date.min_date, np.cumsum(cnt_min_date.ticker))
plt.title('Number of Daily OHLC Values')
plt.show()

In [ ]:
print('The latest date available is {} and it contains {:,.0f} stocks and {:,.0f} ETFs'.format(
    cnt_stock.index[-1], cnt_stock[-1], cnt_etf[-1]))

It looks like the amount of data points is generally growing and it contains a couple of extreme jumps. Given that the overall number of listed companies has been going down this most likely implies that the original download process was being actively extended to include additional tickers.

In particular there is a huge jump taking place sometime in 2005 -- such a sharp jump would indicate that a process collecting data was extended to include a much larger universe of stocks and ETFs.

Let's calculate daily percentage change in the number of per day tickers and check for anomalies.

In [ ]:
cnt_total = cnt_etf.add(cnt_stock, fill_value=0)
cnt = pd.DataFrame({
    'date': cnt_total.index,
    'total': cnt_total.values
},
                   index=cnt_total.index)

cnt['etf'] = cnt_etf
cnt['etf'] = cnt['etf'].fillna(0)

cnt['stock'] = cnt_stock
cnt['stock'] = cnt['stock'].fillna(0)

cnt['stock_cnt_pct_chg'] = np.abs(cnt['stock'].pct_change())
cnt['etf_cnt_pct_chg'] = np.abs(cnt['etf'].pct_change())

In [ ]:
cnt[cnt.stock_cnt_pct_chg > 0.8]

In [ ]:
cnt[cnt.etf_cnt_pct_chg > 0.8]

We can see that the two huge jumps observed on the chart took place on 2005-02-25 and 2011-02-17. Let's take a look at them in detail below.

In [ ]:
cnt['2005-02-23':'2005-02-28']

This jump is probably due to the fact that the collection process was extended to include a much 
bigger universe of stocks and ETFs.

In [ ]:
cnt['2011-02-16':'2011-02-22']

There does not seem to be a significant trading event taking place on this date (that would somehow affect trading for a large number of stocks and ETFs), therefore this drop is likely due to a one-day issue with the data collection process. In fact let's use Yahoo data to test this theory out.

## Comparison with Yahoo Finance Data

In [ ]:
start_date = '2010-01-01'
end_date = '2011-12-31'
example_ticker = 'SPY'

In [ ]:
yh_data = web.DataReader(example_ticker, 'yahoo', start_date, end_date)
yh_data.head()

In [ ]:
csv_data = equity_data.get_csv_data(example_ticker, start_date, end_date)
csv_data.head()

Let's make sure that calls return two identical sets of tradning dates.

In [ ]:
yh_dates_not_csv = yh_data.index[~yh_data.index.isin(csv_data.index)]
if len(yh_dates_not_csv):
    print('Here is a list of dates returned by Yahoo but not found in csv: {}'.
          format(list(yh_dates_not_csv.strftime('%Y-%m-%d'))))

csv_dates_not_yh = csv_data.index[~csv_data.index.isin(yh_data.index)]
if len(csv_dates_not_yh):
    print('Here is a list of dates returned by Yahoo but not found in csv: {}'.
          format(list(csv_dates_not_yh.strftime('%Y-%m-%d'))))

This confirms our earlier theory that the csv data was not fully downloaded on '2011-02-17'.

The csv data is supposed to be split & dividend adjusted as of '2017-11-10' vs how Yahoo adjusts its data based on the the latest available date. Let's download a new set of dates and compare csv prices vs unadjusted Yahoo prices. Let's use ratios instead of absolute differences to perform this comparison.

In [ ]:
start_date = '2016-01-01'
end_date = '2017-11-10'

yh_data = web.DataReader(example_ticker, 'yahoo', start_date, end_date)
csv_data = equity_data.get_csv_data(example_ticker, start_date, end_date)

abs(yh_data['Close'] / csv_data['price']).plot()

These steps (for the most part) are very smooth and imply that csv data was dividend adjusted (SPY does not have any splits).

There are more checks that should be performed to ensure data quality of csv files, but let's stop here for now.

## Utilizing Dask for Analyzing Data

Given that each ticker is saved into its own file, a more efficient (from usability perspective) would be to calculate basic statistics utilizing a tool like Dask. Here is an example how we could go about in case of analyzing stock data.

In [ ]:
# We map stock files to one virtual dataframe. A new column ('path') is created
# which contains a source filename for each data row.

STOCK_DATA_DIR = os.path.join(DATA_DIR, STOCK_DIR)
stocks = dd.read_csv(os.path.join(STOCK_DATA_DIR,
                                  '*' + EquityData.FILE_SUFFIX),
                     include_path_column=True)

# We extract a ticker using the path variable
stocks['ticker'] = stocks['path'].str.replace('.*' + STOCK_DATA_DIR + '/',
                                              '',
                                              regex=True)
stocks['ticker'] = stocks['ticker'].str.replace('.us.txt', '')

stocks = stocks.drop(columns=['Open', 'High', 'Low', 'OpenInt', 'path'])
stocks = stocks.rename(columns={
    'Date': 'date',
    'Close': 'price',
    'Volume': 'volume'
})
stocks.index = stocks.date
stocks = stocks[['ticker', 'price', 'volume']]
stocks.head()

In [ ]:
cnt_stock_dask = stocks.groupby(['date'])['price'].count().compute()

# We need to make sure that index values are sorted
cnt_stock_dask = cnt_stock_dask.sort_index()

In [ ]:
if all(cnt_stock == cnt_stock):
    print('Results for both approaches (Dask & Pandas) are matching up - good')

**Conclusion:** In general, using Dask is much more convenient compared to writing custom functions to process data for each ticker and than aggregate results. One should run some performance checks to further examine pros and cons of using Dask.